# Settings

In [1]:
import numpy as np
import librosa as lr
import matplotlib.pyplot as plt
import soundfile as sf
import sys
sys.path.append("C:/Users/HouseW/LocalPython/SATEST-main/SATEST-main/utils.py")
from utils import FrameExtractor, LPC, ref_derbin, derbin, auto_corr

# plt.rcParams["figure.figsize"] = (5,5)
# plt.rc('font', size=20)
# plt.rc('axes', labelsize=25)
# plt.rc('xtick', labelsize=25)
# plt.rc('ytick',labelsize=25)
# plt.rc('legend', fontsize=20)
# plt.rc('figure', titlesize=50)
# plt.rc('figure', autolayout=True)

sr = 16000
win_time = 0.02
dftlen = 512
# Window Sample의 길이 표현
win_len = int(win_time * sr)
hop_len = int(win_len * 0.5)
print("Window Len:",win_len)
print("Hop Len:",hop_len)

file_path = 'sa0_new.wav'
yr,ori_sr = lr.load(file_path,sr=sr)
time = np.linspace(0,len(yr),len(yr),endpoint=False)/sr




Window Len: 320
Hop Len: 160


## Windowing and Overlapping

In [2]:
# ## Overlap Test

# h = lr.filters.get_window(window="hann", Nx=50)
# z = np.zeros_like(h)
# h = np.concatenate([h,z])
# h2 = np.roll(h,25)
# plt.plot(h)
# plt.plot(h2)
# plt.show()

# # h3 = h+h2+np.roll(h2,25)
# h3 = h+h2
# # print(h3)
# plt.plot(h3)
# plt.show()

# ### OLA를 하려면 hamming은 1.08배 scale이 되어야 하는구나!

In [3]:
# ###
# ### Read input waves
# # file_path = "yonseicrop.wav"
# file_path = 'sa0_new.wav'

# ### Read Audio Files
# yr,ori_sr = lr.load(file_path,sr=sr)
# # print(f"Loaded: {file_path}, Shape: {np.array(yr).shape}, Original sr: {ori_sr}")

# # Time axis
# time = np.linspace(0,len(yr),len(yr),endpoint=False)/sr
# plt.plot(time,yr)
# plt.xlim(0,time[-1])
# plt.show()

In [4]:
# ### Frame Extractor
# ### Extract frames by windowing
# ## 겹치는 Frame을 overlap 해도 원신호에 영향이 가지 않는다

# FE = utils.FrameExtractor(yr,win_len,hop_len)
# frame_arr = FE.extract_frames(win_type="hann")

# # plt.plot(frame_arr[0])
# # plt.plot(frame_arr[1])
# # plt.show()

# x = np.concatenate([frame_arr[0],np.zeros_like(frame_arr[0])])
# y = np.concatenate([frame_arr[1],np.zeros_like(frame_arr[1])])
# y = np.roll(y,hop_len)
# z = x+y / 1.08

# plt.plot(z)
# # plt.show()
# # plt.plot(yr[:win_len*2],'--')
# plt.title("Reconstructed and Original")
# plt.show()

# Frame Visualization In Total

In [5]:
# ## 0번 frame 부터 5개를 표시해보자
# """ Frame Visualization Between Total """

# ind1 = 0
# ind2 = 30
# fig, axes = plt.subplots(5,1,figsize=(15,20))

# time = np.arange(0, len(yr)/sr, 1/sr)
# for i in range(5):
#     startind1 = FE.hop_len * ind1
#     endind1 = FE.hop_len * ind1 + FE.win_len
#     startind2 = FE.hop_len * ind2
#     endind2 = FE.hop_len * ind2 + FE.win_len
#     print("startind:", startind1)
#     frame1 = frame_arr[ind1]
#     frame2 = frame_arr[ind2]
#     ind1 += 1
#     ind2 += 1

#     # 기존 waveform 그리기
#     axes[i].plot(time, yr, label='waveform')
    
#     # 추출된 frame 그리기
#     axes[i].plot(time[startind1: endind1], frame1, label='frame {}'.format(2*i + 1), linestyle='--')
#     axes[i].plot(time[startind2: endind2], frame2, label='frame {}'.format(2*i + 2), linestyle='--')
    
#     # axes[i].set_xlabel('Time(s)', fontsize=18, fontweight='bold')
#     axes[i].set_ylabel('Amplitude', fontsize=14, fontweight='bold')
#     axes[i].set_title('Frame index: {},{}'.format(ind1, ind2), fontsize=16, fontweight='bold')
#     axes[i].tick_params(axis='both', labelsize=20)
#     axes[i].set_xlim(0, time[-1])

#     # axes[i].legend(fontsize=20,loc='lower left')
#     axes[i].grid()
    
#     # plt.figure(figsize=[15,6])
#     # plt.subplot(2,2,1)
#     # plt.plot(frame1)
#     # plt.grid()
#     # plt.xlim(0,len(frame1)-1)
#     # plt.title("Unvoiced")
#     # plt.xlabel("Samples")
#     # plt.subplot(2,2,2)
#     # plt.plot(frame2)
#     # plt.grid()
#     # plt.xlim(0,len(frame2)-1)
#     # plt.title("voiced")
#     # plt.xlabel("Samples")

# # plt.tight_layout()  # subplot 간의 간격 조절
# plt.show()


In [6]:
# ## 각 frame 에서의 autocorrelation 계산 해보기

# ## Auto Correlation Sequence with signal length
# def auto_corr(signal):
#     corr = np.correlate(signal, signal, mode='full')
#     corr = corr[len(corr)//2:]
#     return corr 

# # 각 frame 속에서 p차 LPC 계수의 계수를 반환한다
# def LPC(frame, order=10):
#     coeff_arr = np.zeros(order)
#     # error
#     if len(frame) < order:
#         print('frame is longer than order')
#         return -1
#     # Tx = b
#     ac = auto_corr(frame)[:order]
#     mat_T = make_toeplitz(ac)
#     vec_b = auto_corr(frame)[1:order+1]
#     coeff_arr  = np.dot(np.linalg.inv(mat_T),vec_b)
#     return coeff_arr

# # Make Toeplitz Matrix using Auto Correlation
# def make_toeplitz(ac):
#     p = len(ac)
#     toeplitz_mat = np.zeros((p,p))
#     ac_flip = ac[::-1][:-1]
    
#     for i in range(p):
#         toeplitz_mat[i,:] = np.concatenate((ac_flip[p-i-1:],ac[:p-i]))
        

#     return toeplitz_mat

# # Derbin's Algorithm (As a reference)
# def ref_derbin(r, order):
    
#     # r : 1-D auto corr array
#     a = np.zeros((order+1,order+1))
    
#     # store prediction error for each step
#     E = np.zeros(order+1)
    
#     # First coeff
#     a[0][0] = 1
    
#     # Initial prediction error : power
#     E[0] = r[0]
    
#     # iterate from 1 to order p 
#     for i in range(1,order+1):
#         sum_j = sum(a[i-1][j] * r[i-j] for j in range(1,i))
#         k_i = (r[i] - sum_j ) / E[i-1]
        
#         # Update coefficeints for current step
#         a[i][i] = k_i
#         for j in range(1,i):
#             a[i][j] = a[i-1][j] - k_i * a[i-1][i-j]
            
#         #Update Error
#         E[i] = (1-k_i**2) * E[i-1]
#         # print("i={}, ki={}".format(i,k_i))
#     # Extract final coeff, exclude a0    
#     coeff = a[order][1:]
#     return coeff,E

# # My Derbin's Algorithm
# def derbin(r, p):
#     E = np.zeros(p+1)
#     a = np.zeros((p+1,p+1))
    
#     a[0][0] = 1
#     E[0] = r[0]

#     for i in range(1,p+1):
#         ## sigma
#         j=1
#         sumj = 0
#         while(j <= i-1):
#             sumj+=a[i-1][j]*r[i-j]
#             j += 1
        
#         k_i = (r[i] - sumj) / E[i-1] 
#         a[i][i] = k_i

#         ## i-order 새로운 coeff 갱신
#         for j in range(1,i):
#             a[i][j] = a[i-1][j] - k_i * a[i-1][i-j]
            
#         E[i] = (1 - k_i**2)*E[i-1]
#         coeff = a[p][1:]
#     return coeff,E



# # 예제 자동 상관 함수
# auto_corr_sequence = np.array([0, 1, 2, 3,4,5])

# # Toeplitz 행렬 생성
# # toeplitz_result = make_toeplitz(auto_corr_sequence)
# # print(LPC([1,2,34,5,6,7,8],order=6))

# # print(len(frame_arr))

# ########################## LPC on every frames
# # frame_len = len(frame_arr)
# # coeff_arr = []
# # # print(frame_len, len(coeff_arr))
# # for ind,frame in enumerate(frame_arr):
# #     # 매 frame에서 LPC 계수 계산
# #     coeff = LPC(frame,order=10)
# #     coeff_arr.append(coeff) 
# #     break
# y = frame_arr[30]

# derbin(auto_corr(y),3)
# print("\n")
# derbin(auto_corr(y),3)

In [7]:
# ### librosa의 LPC와 비교하는 것

# import matplotlib.pyplot as plt
# import scipy
# y, sr = lr.load(lr.ex('libri1'), duration=0.020)

# y = yr
# s = 16000
# a = lr.lpc(y, order=5)
# b = np.hstack([[0], -1 * a[1:]])
# print(a)
# print(b)

# mya = LPC(y, order=5)
# myb = np.hstack([[0],mya[:]])
# print(mya)
# print(myb)

# myy = scipy.signal.lfilter(myb, [1], y)
# y_hat = scipy.signal.lfilter(b, [1], y)
# fig, ax = plt.subplots()
# ax.plot(y)
# # ax.plot(y_hat, linestyle='--')
# ax.plot(myy ,label="mine")
# ax.legend(['y', 'y_hat'])
# ax.set_title('LP Model Forward Prediction')

In [8]:
### librosa의 LPC와 비교하는 것
### LPC 계수가 올바르게 추출 되었는지 디버깅
import librosa
import matplotlib.pyplot as plt
import scipy
from utils import FrameExtractor, LPC, ref_derbin, derbin, auto_corr

FE = FrameExtractor(yr,win_len=win_len,hop_len=hop_len)
frame_arr = FE.extract_frames(win_type="rectangular")
y = frame_arr[30]

a = librosa.lpc(y, order=3)
print(a)
mya = LPC(y, order=3)
drbina,e1 = ref_derbin(auto_corr(y),order=3)
mydrbina,e2 = derbin(auto_corr(y), p=3)

print("librosa LPC: \t\t", -1*a[1:])
print("Matrix Inverse: \t", mya)
print("Derbin's Inverse: \t", drbina)
print("myDerbin's Inverse: \t", mydrbina)

rectangular
From 8200 samples, total 50 frames are generated
[ 1.0000000e+00 -1.8354051e+00  9.0559781e-01 -6.7733682e-04]
librosa LPC: 		 [ 1.8354051e+00 -9.0559781e-01  6.7733682e-04]
Matrix Inverse: 	 [ 1.5108497  -0.37836478 -0.22113916]
Derbin's Inverse: 	 [ 1.5108497  -0.37836478 -0.22113916]
myDerbin's Inverse: 	 [ 1.5108497  -0.37836478 -0.22113916]


In [ ]:
##
## Filter의 Vocal Tract Response 추정해보기

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter, freqz, hamming, lfilter_zi, lfiltic
from scipy.linalg import toeplitz, inv

## GPT의 LPC 
def lpc_analysis(signal, order=10):
    """주어진 신호에 대한 LPC 계수를 계산합니다."""
    # 자기상관 함수 계산
    autocorr = np.correlate(signal, signal, mode='full')
    autocorr = autocorr[len(autocorr)//2:]

    # 토플리츠 행렬 및 벡터 준비
    R = toeplitz(autocorr[:order])
    
    r = autocorr[1:order+1]
    
    # LPC 계수 계산 (Levinson-Durbin 알고리즘)
    lpc_coeffs = np.dot(inv(R), r)
    lpc_coeffs = np.insert(lpc_coeffs, 0, 1)  # LPC 계수에 1 추가
    return lpc_coeffs

# 예제 신호 생성 (예: 사인파)
fs = 8000  # 샘플링 주파수
t = np.linspace(0, 1, fs)
signal = np.sin(2 * np.pi * 440 * t) + 0.5 * np.sin(2 * np.pi * 880 * t)
signal = frame_arr[25]

# LPC 분석
order = 12  # LPC 계수의 수
a = LPC(signal, order=order)
a = np.concatenate(([1],-a))
print(a)
w, h = freqz(b=[1], a=a)

sr = 16000
# w축을 0-0.5Fs 로 정규화
w = w /np.pi * 0.5 * sr
# 주파수 응답 시각화
plt.figure(figsize=(10, 6))
plt.plot(w, 20 * np.log10(abs(h)), label='Frequency Response')
plt.title('Frequency Response of LPC Filter')
plt.xlabel('Frequency [rad/sample]')
plt.ylabel('Amplitude [dB]')
plt.grid(True)
plt.legend()
plt.show()

ff = np.fft.fft(signal,1024)
ff = np.fft.fftshift(ff)
ff = ff[len(ff)//2:]
# ff = np.abs(ff)
plt.figure(figsize=(10, 6))
plt.plot(w,20 * np.log10(abs(ff)))
plt.grid()
plt.title("FFT of window")
plt.show()


print(len(ff))
print(len(w))

In [ ]:
## 주어진 input 신호와, 에러 신호를 그려봅시다
## Error 신호 그리기

signal = frame_arr[30]
order = 10

# LPC Analysis
coeff = LPC(signal, order=10)

# Error 전달함수의 계수
coeff_T = np.concatenate(([1],-coeff))
# tt = lr.lpc(signal,order=order)
# print(tt)
print(coeff_T)

filtered = lfilter(coeff_T,[1],signal)

# 입력 신호와 에러 신호(필터링된 신호) 그리기
plt.figure(figsize=(14, 7))

# 입력 신호 플롯
# plt.subplot(2, 1, 1)
plt.plot(signal, label='Input Signal')
plt.title('Input Signal')
plt.grid(True)
plt.xlim(0,len(signal)-1)
plt.legend()

# 에러 신호(필터링된 신호) 플롯
# plt.subplot(2, 1, 2)
plt.plot(filtered, label='Filtered Signal (Error Signal)')
plt.title('Filtered Signal (Error Signal)')
plt.grid(True)
plt.legend()
plt.xlim(0,len(signal)-1)
plt.tight_layout()
plt.show()


# 음성 합성


In [ ]:
## 각 frame에서 해야할 일

"""
LPC 계수 측정하기
Pitch 측정하기 - voiced
Unvoiced 판단하기
"""

signal = frame_arr[30]
# plt.plot(signal)
# plt.show()
### Frame에서 진행하는 Pitch Detection

## 900Hz Lowpass Filering
cutoff_freq = 900.0
sr = 16000
nyquist = 0.5 * sr
num_taps = 101 # 필터의 길이

#FIR LPF
lpf_coeff = scipy.signal.firwin(num_taps, cutoff=cutoff_freq, fs=sr, pass_zero='lowpass', window="hamming")
yr_lpf = scipy.signal.lfilter(lpf_coeff, 1.0, signal)

# plt.plot(yr_lpf)
# plt.show()

# ###################### 주파수 스펙트럼 plot
# filtered_signal = lfilter(lpf_coeff, 1.0, signal)

# # 주파수 스펙트럼 계산
# fft_original = np.fft.fft(signal)
# fft_filtered = np.fft.fft(filtered_signal)
# freq = np.fft.fftfreq(len(signal), 1/fs)

# # 주파수 스펙트럼 시각화
# plt.figure(figsize=(12, 6))
# plt.plot(freq, np.abs(fft_original), label='Original Signal')
# plt.plot(freq, np.abs(fft_filtered), label='Filtered Signal', linestyle='--')
# plt.title('Frequency Spectrum')
# plt.xlabel('Frequency (Hz)')
# plt.ylabel('Amplitude')
# plt.xlim(0, fs/2)  # Nyquist 주파수까지 표시
# plt.legend()
# plt.grid(True)
# plt.show()

# plt.plot(signal)
# plt.plot(filtered_signal)
# plt.show()

In [ ]:
## CL Clipping - Center clipping
class ThresholdClipper:
    def __init__(self, function):
        self.function = function
        self.CL = self.calculate_thres()
        self.CL_max = self.calculate_thres_max()
        
        
    def calculate_thres_max(self):
        function = np.abs(self.function)
        CL = 0.4 * np.max(function)
        return CL
        
    def calculate_thres(self):
        function = np.abs(self.function)
        first_max = np.max(function[0:len(function)//3]) 
        last_max = np.max(function[len(function)//3 * 2:])
        CL = 0.68 * min(first_max,last_max)
        return CL
        
    def center_clip(self,CL):
        function = self.function
        y = np.zeros_like(function)
        for n in range(0,len(y)):
            val = function[n]
            if val >= CL:
                y[n] = val - CL
            elif val <= (-1*CL):
                y[n] = val + CL
            else:
                y[n] = 0
        return y
    
    def infinite_clip(self,CL):
        function = self.function
        y = np.zeros_like(function)
        for n in range(0,len(y)):
            val = function[n]
            if val >= CL:
                y[n] = 1
            elif val <= (-1*CL):
                y[n] = -1
            else:
                y[n] = 0
        return y        

In [ ]:
## 각 frame에서 해야할 일

"""
LPC 계수 측정하기
Pitch 측정하기 - voiced
Unvoiced 판단하기
"""
cutoff_freq = 900.0
sr = 16000
order = 10
for signal in frame_arr[15:20]:
    
    # signal = frame_arr[30]
    ### Frame에서 진행하는 Pitch Detection

    ## 900Hz Lowpass Filering
    #FIR LPF
    lpf_coeff = scipy.signal.firwin(numtaps=101, cutoff=cutoff_freq, fs=sr, pass_zero='lowpass', window="hamming")
    signal_lpf = scipy.signal.lfilter(lpf_coeff, 1.0, signal)

    # plt.plot(signal_lpf)
    # plt.show()

    ## Frame 에서 LPC 계수 계산하기
    coeff = LPC(signal_lpf, order=order)
    print(coeff)

    ## Clipping 적용하기
    Clipper = ThresholdClipper(signal_lpf)
    signal_clipped = Clipper.center_clip(Clipper.CL)

    # AC 계산하기
    ac_arr = auto_corr(signal_clipped)

    # Enery
    energy = ac_arr[0]
    voice_thres = energy * 0.4

    # Find Peaks of AC 
    peakval = np.max(ac_arr)    
    maxima_indices, _ = scipy.signal.find_peaks(ac_arr)
    maxima_indices = maxima_indices[maxima_indices>50]
    
    if maxima_indices.size > 0:
        maxval = np.max([ac_arr[i] for i in maxima_indices])
        idx = np.argmax([ac_arr[i] for i in maxima_indices])
        max_idx = maxima_indices[idx]
    else:   
        maxval = 0
    flag = 1 if maxval > voice_thres else 0
    pitch_freq = sr / max_idx if flag else 0
    
    print(pitch_freq)
    plt.plot(ac_arr)
    plt.show()

In [ ]:
# Pitch Detector 함수 만들기
# voiced_flag : 1 if voiced, else 0
# pitch : pitch frequency if voiced, else 0
    
def PitchDetector(signal, sr=16000):
    ## Clipping 적용하기
    Clipper = ThresholdClipper(signal)
    signal_clipped = Clipper.center_clip(Clipper.CL)

    # AC 계산하기
    ac_arr = auto_corr(signal_clipped)

    # plt.plot(ac_arr)
    # plt.show()

    # Enery
    energy = ac_arr[0]
    voice_thres = energy * 0.35

    # Find Peaks of AC 
    peakval = np.max(ac_arr)    
    maxima_indices, _ = scipy.signal.find_peaks(ac_arr)
    maxima_indices = maxima_indices[maxima_indices>50]
    
    # print(maxima_indices)
    if maxima_indices.size > 0:
        maxval = np.max([ac_arr[i] for i in maxima_indices])
        idx = np.argmax([ac_arr[i] for i in maxima_indices])
        max_idx = maxima_indices[idx]
        # print(maxval, voice_thres)
        voiced_flag = 1 if maxval > voice_thres else 0
        pitch = sr / max_idx if voiced_flag else 0
        
    else:
        voiced_flag = 0
        pitch = 0

    return voiced_flag, pitch

In [ ]:
## 각 frame에서 해야할 일

"""
LPC 계수 측정하기
Pitch 측정하기 - voiced
Unvoiced 판단하기
"""
cutoff_freq = 900.0
sr = 16000
order = 10

file_path = "yonseicrop.wav"
yr,_ = lr.load(file_path,sr=sr)

## LPF to yr
lpf_coeff = scipy.signal.firwin(numtaps=101, cutoff=cutoff_freq, fs=sr, pass_zero='lowpass', window="hamming")
yr_lpf = scipy.signal.lfilter(lpf_coeff, 1.0, yr)

FE = utils.FrameExtractor(yr_lpf,win_len,hop_len)
# Pitch Detection 에서는 rectangular를 써야할 듯
frame_arr = FE.extract_frames(win_type="rectangular")
ham_arr = FE.extract_frames(win_type="hamming")

i=0
pitchlist = []
## 20번 frame을 예로 들어봅시다
for i, signal in enumerate(frame_arr[:]):   
    # signal = frame_arr[30]
    ### Frame에서 진행하는 Pitch Detection

    # plt.plot(signal)
    # plt.show()
    # if i==5:
    #     break
    # i+=1
    
    ## Frame 에서 LPC 계수 계산하기
    coeff = LPC(signal, order=order)

    voiced_flag, pitch = PitchDetector(signal, sr)
    if pitch: 
        print(i, "pitch: {}".format(pitch))
    pitchlist.append(pitch)
    
    ## 20번 frame의 신호를 정확하게 reconstruct 해볼까요?

# meidanfiltering to pitchlist
pitchlist = scipy.signal.medfilt(pitchlist, kernel_size=5)

plt.figure(figsize=(10,6))
plt.plot(pitchlist)
plt.ylim(-1,200)
plt.show()

plt.figure(figsize=(10,6))
plt.plot(np.linspace(0,len(frame_arr),len(yr)), yr)
plt.show()


# LPC Synthesis


In [ ]:

## Plot LPC Spectrum
# 신호의 스펙트럼 플롯
def PlotLPCSpectrum(signal, sr, p=10, dftlen=2048):
    # signal_fft = np.fft.fft(signal)
    # signal_fft = np.fft.rfft(signal,dftlen)[1:]
    freqs = np.linspace(0, sr/2, dftlen//2)
    
    # freqs = np.fft.fftfreq(len(signal_fft), 1/fs)
    
    coeff = LPC(signal, order=order)
    lpc_coeff = np.concatenate(([1],-coeff))
    w, h = scipy.signal.freqz([0.15], lpc_coeff, worN=dftlen//2, fs=fs)
    
    # Energy
    signal_energy = np.sum(np.abs(np.fft.rfft(signal, dftlen//2))**2)
    lpc_energy = np.sum(np.abs(h)**2)  # LPC 스펙트럼의 에너지 계산
    adjust_factor = np.sqrt(signal_energy / lpc_energy)
    
    print("adj:",adjust_factor)
    w2, h2 = scipy.signal.freqz([adjust_factor], lpc_coeff, worN=dftlen//2, fs=sr)
    signal_fft = np.fft.rfft(signal, 4096)
    
    plt.figure(figsize=(15,3))
    # plt.subplot(1, 2, 1)
    plt.plot(freqs, 20 * np.log10(np.abs(signal_fft[:dftlen//2])), label='Original Signal Spectrum')
    plt.title('Signal Frequency Spectrum')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')
    plt.xlim(0, sr//2)
    plt.grid(True)

    # print(len(freqs),len(np.abs(h)))
    # LPC 필터의 주파수 응답 플롯
    # plt.subplot(1, 2, 2)
    plt.plot(freqs, 20 * np.log10(np.abs(h2)), label='LPC Filter Frequency Response')
    plt.title('LPC Filter Frequency Response')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain (dB)')
    plt.grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
#3 Plot LPC
signal = frame_arr[5]
PlotLPCSpectrum(signal, sr=sr, p=order)

# SYNTHESIZE

In [ ]:
## Reconstruction 20번 frame
## 각 frame에서 해야할 일

"""
LPC 계수 측정하기
Pitch 측정하기 - voiced
Unvoiced 판단하기
"""
cutoff_freq = 900.0
sr = 16000
order = 15

file_path = "yonseicrop.wav"
yr,_ = lr.load(file_path,sr=sr)

## LPF to yr
lpf_coeff = scipy.signal.firwin(numtaps=101, cutoff=cutoff_freq, fs=sr, pass_zero='lowpass', window="hamming")
yr_lpf = scipy.signal.lfilter(lpf_coeff, 1.0, yr)
FE = utils.FrameExtractor(yr_lpf,win_len,hop_len)

# Pitch Detection 에서는 rectangular를 써야할 듯
frame_arr = FE.extract_frames(win_type="rectangular")
signal = frame_arr[77]

#######################################################
# Pitch
voiced_flag, pitch = PitchDetector(signal, sr)
if pitch: 
    print(i, "pitch: {}".format(pitch))
    
## Frame 에서 LPC 계수 계산하기
## 뽑아낸 피치 : 115 을 이용해서 impulse를 구축해볼까?
# 8ms 단위의 impulse
# 139 샘플?

coeff = LPC(signal, order=order)

if pitch: 
    duration = 0.008  # impulse 간격(초), 예시로 8ms
    samples_per_impulse = int(sr / pitch)
    
    # Impulse Train 초기화, 길이를 window_length로 설정
    impulse_train = np.zeros(len(signal))

    # 각 펄스 위치에 1 설정
    for i in range(0, len(signal), samples_per_impulse):
        if i+50 < len(signal):
            impulse_train[i+50] = 1

    plt.plot(impulse_train)
    plt.show()

    # LPC 계수 (여기서는 예시로 사용, 실제 계산 필요)
    lpc_coeff = np.concatenate(([1],-coeff))

    # LPC 필터를 사용한 신호 합성
    synthesized_signal = scipy.signal.lfilter([1.0], lpc_coeff, impulse_train)

    # 결과 확인
    plt.plot(synthesized_signal)
    plt.title("Synthesized Signal with LPC")
    plt.show()

    print(coeff)
    # filtered = lfilter(coeff,[1],signal)

plt.plot(signal)
plt.title("Frame")
plt.show()



In [ ]:
"""
Definition of Frame Synthesis
"""

## input 으로 frame을 받아, 해당 frame을 반환해보자
def Synthesis(signal, sr, p=10, plotopt = False):
    #######################################################
    window = lr.filters.get_window("hann",Nx=len(signal))
    rectsignal = signal
    signal = signal * window
    
    coeff = LPC(signal, order=p)
    lpc_coeff = np.concatenate(([1],-coeff))
    
    # Pitch Detection
    voiced_flag, pitch = PitchDetector(rectsignal, sr)
    # print("signal is {}".format(voiced_flag))
    
    if voiced_flag: 
        # print("pitch: {:.2f}Hz".format(pitch))
        
        duration = 0.008  # impulse 간격(초), 예시로 8ms
        samples_per_impulse = int(sr / pitch)
        
        # Impulse Train 초기화, 길이를 window_length로 설정
        impulse_train = np.zeros(len(signal))

        # 각 펄스 위치에 1 설정
        n = 200
        for i in range(0, len(signal), samples_per_impulse):
            if i+n < len(signal):
                impulse_train[i+n] = 1

        # plt.plot(impulse_train)
        # plt.show()

        # LPC 필터를 사용한 신호 합성
        synthesized_signal = scipy.signal.lfilter([1.0], lpc_coeff, impulse_train)

    else : # Unvoiced
        sig_std = np.std(signal)
        excitation = np.random.normal(loc=0.0, scale = sig_std, size = signal.shape)
        
        F_excitation = np.fft.fft(excitation, len(signal))
        w, h = scipy.signal.freqz([1],lpc_coeff,worN=len(signal),whole=True, fs=sr)
        F_result = F_excitation*h
        synthesized_signal = np.fft.ifft(F_result, len(signal))
        synthesized_signal = np.real(synthesized_signal)
    
    # print("sythn length:{}".format(len(synthesized_signal)))
    
    if plotopt:
        plt.figure(figsize=(15,3))
        plt.subplot(1,2,1)
        plt.plot(synthesized_signal)
        plt.title("Synthesized Signal with LPC")

        # print(coeff)
        # filtered = lfilter(coeff,[1],signal)
        plt.subplot(1,2,2)
        plt.plot(signal)
        plt.title("Frame")
        plt.show()
        
    return synthesized_signal

    # 결과 확인
    PlotLPCSpectrum(signal, sr=sr, p=order)


In [ ]:
## SYNTHESIZE
file_path = "sa0_new.wav"
file_path = "yonseicrop.wav"
y,__ = lr.load(file_path,sr=sr)
FE = utils.FrameExtractor(y,win_len,hop_len)
frame_arr = FE.extract_frames(win_type="rectangular")
synthesized = np.zeros_like(y)

for i, signal in enumerate(frame_arr[:]):
    tempsig = np.zeros_like(y)
    # print("num: {}".format(i))
    start = i * hop_len
    end = start + win_len
    tempsig[start:end]=Synthesis(signal, sr=sr,p=8,plotopt=False)
    synthesized = synthesized + tempsig

plt.figure(figsize=(15,3))
plt.subplot(1,2,1)
plt.plot(synthesized)
plt.subplot(1,2,2)
plt.plot(y)
plt.show()

# save_path = "synthesized.wav"
# # print(save_path)
# sf.write(save_path, synthesized, sr)

import IPython.display as ipd
ipd.Audio(synthesized, rate=sr,autoplay=True)


In [ ]:
# Load Data
import IPython.display as ipd
ipd.Audio(yr, rate=sr, autoplay=True)

# save_path = "synthesized.wav"
# # print(save_path)
# sf.write(save_path, synthesized, sr)